### Menu Multi-Agent Supervisor

Creates an Agent Bricks Supervisor Agent that coordinates:
- **Genie space** - for structured data queries (nutrition stats, allergen filtering, price comparisons, inspection scores)
- **Menu Knowledge Assistant** - for document Q&A over the menu PDFs
- **Inspection Knowledge Assistant** - for document Q&A over the food safety inspection reports

In [ ]:
%pip install --upgrade databricks-sdk
dbutils.library.restartPython()

In [ ]:
CATALOG = dbutils.widgets.get("CATALOG")
KNOWLEDGE_ENDPOINT = dbutils.widgets.get("MENU_KNOWLEDGE_ENDPOINT_NAME")
INSPECTION_ENDPOINT = dbutils.widgets.get("INSPECTION_KNOWLEDGE_ENDPOINT_NAME")
SUPERVISOR_ENDPOINT = dbutils.widgets.get("MENU_SUPERVISOR_ENDPOINT_NAME")

##### Retrieve the Genie space ID from uc_state

In [ ]:
import json

state_df = spark.sql(f"""
    SELECT resource_data
    FROM {CATALOG}._internal_state.resources
    WHERE resource_type = 'genie_spaces'
    ORDER BY created_at DESC
    LIMIT 1
""")

if state_df.count() > 0:
    genie_info = json.loads(state_df.first().resource_data)
    GENIE_SPACE_ID = genie_info.get("space_id", "")
    print(f"Found Genie space: {GENIE_SPACE_ID}")
else:
    raise RuntimeError("No Genie space found in uc_state. Ensure menu_genie stage ran first.")

##### Create the Supervisor Agent

In [ ]:
from databricks.sdk import WorkspaceClient
import json

w = WorkspaceClient()

def get_endpoint_id_by_name(endpoint_name):
    """Resolve a serving endpoint name to its ID via the /api/2.0/tiles listing."""
    resp = w.api_client.do("GET", "/api/2.0/tiles")
    for tile in resp.get("tiles", []):
        if tile.get("name") == endpoint_name:
            return tile["serving_endpoint_name"]
    raise RuntimeError(
        f"No tile found with name '{endpoint_name}'. "
        f"Ensure the knowledge agent stage completed successfully."
    )

MENU_KA_NAME = f"{CATALOG}-menu-knowledge"
INSPECTION_KA_NAME = f"{CATALOG}-inspection-knowledge"

KNOWLEDGE_ENDPOINT_ID = get_endpoint_id_by_name(MENU_KA_NAME)
print(f"Resolved menu knowledge endpoint: {MENU_KA_NAME} -> {KNOWLEDGE_ENDPOINT_ID}")

INSPECTION_ENDPOINT_ID = get_endpoint_id_by_name(INSPECTION_KA_NAME)
print(f"Resolved inspection knowledge endpoint: {INSPECTION_KA_NAME} -> {INSPECTION_ENDPOINT_ID}")

AGENT_NAME = f"{CATALOG}-menu-supervisor"
API_BASE = "/api/2.0/multi-agent-supervisors"

SUPERVISOR_BODY = {
    "name": AGENT_NAME,
    "description": (
        "Caspers Kitchens assistant that coordinates structured data queries, "
        "menu document search, and food safety compliance across 16 restaurant brands "
        "and 4 ghost kitchen locations. Routes analytics to the Genie space, menu "
        "questions to the Menu Knowledge Assistant, and safety/compliance questions "
        "to the Inspection Knowledge Assistant."
    ),
    "endpoint_name": SUPERVISOR_ENDPOINT,
    "agents": [
        {
            "agent_type": "genie",
            "genie_space": {"id": GENIE_SPACE_ID},
            "name": "menu-safety-analytics",
            "description": (
                "Queries the structured database for nutrition stats, allergen "
                "filtering, price comparisons, brand-level analytics, inspection "
                "scores, violation counts, and compliance summaries. Use for "
                "questions that need numbers, counts, comparisons, or filtering."
            ),
        },
        {
            "agent_type": "ka",
            "serving_endpoint": {"name": KNOWLEDGE_ENDPOINT_ID},
            "name": "menu-document-search",
            "description": (
                "Searches the original restaurant menu PDF documents for dish "
                "descriptions, preparation details, ingredient information, and "
                "specific menu content. Use for questions about what's on a menu "
                "or details about specific dishes."
            ),
        },
        {
            "agent_type": "ka",
            "serving_endpoint": {"name": INSPECTION_ENDPOINT_ID},
            "name": "inspection-report-search",
            "description": (
                "Searches food safety inspection report PDFs for violation details, "
                "corrective actions, inspector findings, and compliance status. "
                "Use for questions about health inspections, violations, food safety "
                "compliance, and corrective action deadlines at specific locations."
            ),
        },
    ],
    "instructions": (
        "You are a helpful assistant for Caspers Kitchens covering menus and food safety. "
        "Always cite which brand or location the information comes from. "
        "Be specific about allergen warnings, nutritional details, and violation severities. "
        "Use Menu & Safety Analytics for structured queries (counts, averages, filtering), "
        "Menu Document Search for descriptive questions about dishes, and "
        "Inspection Report Search for food safety and compliance questions."
    ),
}

def find_existing_id():
    """Look up supervisor ID from uc_state."""
    try:
        df = spark.sql(f"""
            SELECT resource_data FROM {CATALOG}._internal_state.resources
            WHERE resource_type = 'endpoints'
            ORDER BY created_at DESC
        """)
        for row in df.collect():
            info = json.loads(row.resource_data)
            if info.get("endpoint_name") == SUPERVISOR_ENDPOINT:
                return info.get("agent_id")
    except Exception:
        pass
    return None

def try_get(agent_ref):
    """GET the supervisor by ID or name, return the response or None."""
    try:
        return w.api_client.do("GET", f"{API_BASE}/{agent_ref}")
    except Exception:
        return None

existing_id = find_existing_id()
agent_id = None
needs_polling = True

# Path 1: found ID in uc_state — verify and update
if existing_id:
    info = try_get(existing_id)
    if info:
        print(f"Supervisor exists ({existing_id}), updating...")
        w.api_client.do("PUT", f"{API_BASE}/{existing_id}", body=SUPERVISOR_BODY)
        agent_id = existing_id
        needs_polling = False
        print(f"\u2705 Updated Supervisor Agent: {agent_id}")

def extract_agent_id(resp):
    """Extract agent ID from API response, trying common key names."""
    for key in ("id", "agent_id", "supervisor_id", "name"):
        val = resp.get(key)
        if val:
            return val
    return None

# Path 2: no ID in state (or GET failed) — try to create
if not agent_id:
    try:
        supervisor = w.api_client.do("POST", API_BASE, body=SUPERVISOR_BODY)
        print(f"POST response keys: {list(supervisor.keys())}")
        agent_id = extract_agent_id(supervisor)
        print(f"\u2705 Created Supervisor Agent: {agent_id}")
    except Exception as e:
        err = str(e)
        if "already exists" in err.lower() or "ALREADY_EXISTS" in err:
            print(f"Supervisor already exists, looking up by name: {AGENT_NAME}")
            info = try_get(AGENT_NAME)
            if info:
                agent_id = extract_agent_id(info)
                if agent_id:
                    print(f"Found by name ({agent_id}), updating...")
                    w.api_client.do("PUT", f"{API_BASE}/{agent_id}", body=SUPERVISOR_BODY)
                    needs_polling = False
                    print(f"\u2705 Updated Supervisor Agent: {agent_id}")
                else:
                    agent_id = AGENT_NAME
                    needs_polling = False
                    print(f"Supervisor {AGENT_NAME} already exists and is running. Proceeding.")
            else:
                agent_id = AGENT_NAME
                needs_polling = False
                print(f"Supervisor {AGENT_NAME} already exists and is running. Proceeding.")
        else:
            raise

# Path 3: POST succeeded but returned no recognizable ID — resolve by name
if not agent_id:
    print(f"agent_id is still None after creation. Attempting name-based lookup: {AGENT_NAME}")
    info = try_get(AGENT_NAME)
    if info:
        agent_id = extract_agent_id(info)
        print(f"Resolved via name lookup: {agent_id}")
    if not agent_id:
        agent_id = AGENT_NAME
        needs_polling = False
        print(f"Using agent name as identifier: {agent_id}")

In [ ]:
import time

if needs_polling:
    if not agent_id:
        print(f"⚠️ No agent_id available — skipping polling. Check POST response above.")
    else:
        MAX_WAIT = 300
        POLL_INTERVAL = 30
        elapsed = 0
        print(f"Checking if Supervisor endpoint is ready (agent_id={agent_id}, max {MAX_WAIT}s)...")

        while elapsed < MAX_WAIT:
            try:
                sup_status = w.api_client.do("GET", f"{API_BASE}/{agent_id}")
                state = sup_status.get("endpoint_status", "")
                print(f"  endpoint_status: {state}")
                if str(state).upper() in ("ACTIVE", "READY", "ONLINE"):
                    print(f"\u2705 Supervisor {AGENT_NAME} is READY")
                    break
            except Exception as e:
                print(f"  GET status check failed: {type(e).__name__}: {e}")

            time.sleep(POLL_INTERVAL)
            elapsed += POLL_INTERVAL
        else:
            print(f"\u2705 Endpoint may still be provisioning — proceeding.")
else:
    print(f"\u2705 Supervisor already running — skipping polling.")

In [ ]:
import sys
sys.path.append('../utils')
from uc_state import add

add(CATALOG, "endpoints", {"endpoint_name": SUPERVISOR_ENDPOINT, "agent_id": agent_id})
print("\u2705 Menu Supervisor stage complete")